In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
os.environ['TRANSFORMERS_CACHE'] = '/mnt/iMVR/junde/.cache/huggingface/hub'

from langchain.llms import HuggingFacePipeline
import networkx as nx
from transformers import pipeline
import re
from creat import creat_world
from prompt import *

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipeline = pipeline(model="meta-llama/Llama-2-7b-chat-hf", device_map="auto")
model = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.60s/it]


In [3]:
locations, agents = creat_world(model)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, sup

In [4]:
global_time = 0
for repeats in range(5):
  global_time += 1
  print("In global time", global_time)
  action_prompts = {}
  action_results = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    # for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
    people_description = []
    for i in people: 
      if i not in action_results: # initialize action results as the observations
        action_results[i] = agents[i].get_summary(force_refresh=False)
      people_description.append(i+': '+ action_results[i])


    for i in people: # add observation to memory and react
      others = [x for x in people if x != i]
      observation = "You are {}.You are currently in {} with the following description: {}. \
      It is currently {}:00. The following people are in this area: {}. You can interact with them.". \
      format(i, location, town_areas[location], str(global_time), ', '.join(others))

      others_des = [x for x in people_description if i+': ' not in x]
      observation += ' You know the following about people: ' + '. '.join(others_des)

      # print("for people", i)
      # print("the observation is:", observation)

      agents[i].memory.add_memory(observation)
      _, reaction = agents[i].generate_reaction(observation)
      action_results[i] = reaction

      print("action result is:")
      print(reaction)

In global time 1
for people Toblen Stonehill
the observation is: You are Toblen Stonehill.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.. Linene Graywind: Name: Linene Graywind (age: 38)
Innate traits: competent, forbearing, self-discipline, impressionable 
Linene runs a trading post.

relevant_mem is: - [December 06, 2023, 11:44 PM] You are Toblen Stonehill.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.. Linene Graywind: Name: Linene Graywind (age: 38)
Innate traits: competent, forbearing, self-discipline, impressionable 
Linene runs a trading post.. Harbin Wester:

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


What would you like to do?
for people Linene Graywind
the observation is: You are Linene Graywind.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Toblen Stonehill, Daran Edermath, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Toblen Stonehill: Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.. Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.. Harbin Wes

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/langchain/schema/vectorstore.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="You are Linene Graywind.You are currently in Phandalin Town Square with the following description: Town square of the town of P

...

relevant_mem is: Toblen Stonehill is a merchant who owns a trading post in Phandalin. He is known for his mean and gossipy nature, and is often seen as arrogant and selfish.
What is the relationship between Terrill Bloodscar and Toblen Stonehill?
Terrill Bloodscar and Toblen Stonehill have a neutral relationship.


/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


What is the result of Terrill Bloodscar's reaction?
In global time 3
for people Toblen Stonehill
the observation is: You are Toblen Stonehill.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 3:00. The following people are in this area: Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.. Linene Graywind: Name: Linene Graywind (age: 38)
Innate traits: competent, forbearing, self-dis

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/gen

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.09 GiB. GPU 0 has a total capacty of 23.68 GiB of which 889.12 MiB is free. Including non-PyTorch memory, this process has 22.79 GiB memory in use. Of the allocated memory 20.42 GiB is allocated by PyTorch, and 2.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF